In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linprog,milp,LinearConstraint
import itertools

### 2.3.3 The scheduling problem and integer programming
In the previous section we looked at the assignment problem, which had a linear programming solution, but also a multiagent solution based on the idea of auctions and competitive equilibrium. Now we'll look at a more complicated scenario where multiple objects can be given to someone. The problem can still be stated as a linear program, but the answers will no longer be integers, hence the need for integer programming.

#### Definition

In a scheduling problem we have a set $N$ of $n$ agents, a set $X$ of $m$ discrete and consecutive time slots, a value function $v$ for assigning a combination of time slots to an agent, and a price $q$ to use each time slot for some other purpose (this is optional).

The value function says that a value of $w_i$ is generated if the number of time slots assigned to agent $i$ is at least $\lambda_i$ (some measure of the time needed) before $d_i$ (the deadline).

Consider this problem, where we have 5 hour-long time slots (9AM,10AM,11AM,12PM,1PM), and no alternative uses:

$
\begin{array}{cclc}
\text{agent} & \text{length}(\lambda) & \text{deadline}(d) & \text{value}(w) \\
\hline
\text{A} & 2 & \text{11 AM} & \$2 \\
\text{B} & 2 & \text{1 PM} & \$5 \\
\text{C} & 1 & \text{2 PM} & \$7 \\
\text{D} & 3 & \text{12 PM} & \$3 \\
\text{E} & 1 & \text{1 PM} & \$4 \\
\end{array}
$

This problem has a solution, which can be worked out by first putting $C$ at the end (as nothing else will be using that last slot) and then seeing out that of the other options the best is $B$ and $E$:

$
\begin{array}{lc}
\text{slot} & \text{user} \\
\hline
\text{9 AM} &  B\\
\text{10 AM} & B\\
\text{11 AM} & \\
\text{12 PM} & E \\
\text{1 PM} &  C \\
\end{array}
$

And there are other solutions, which are essentially just permutations of this. 

But how do we get there algorithmically?

First of all, the problem can be written as an integer program, where $S$ is the set of all possible subsets of $X$, and $x_{i,s}$ refers to the variable saying if agent $i$ was assigned to $s\in S$.

$$
\begin{align*}
\text{maximise} \quad & \sum_{s\in S, i\in N} v_i(s)x_{i,s} \\
\text{subject to} \quad & \sum_{s\in S} x_{i,s}<=1 \quad \forall i\in N \\
\text{} \quad & \sum_{s\in S: j\in X, i\in N} x_{i,s}<=1 \quad \forall j\in X \\
\end{align*}
$$

I.e., each person can only be assigned to 1 set, and each slot can only appear in one assignment. We can see this set up below. 

In [2]:
slots = ["9am","10am","11am","12pm","1pm"]
agents = ["A","B","C","D","E"]
lengths = np.array([2,2,1,3,1])
deadlines = np.array([1,3,4,2,3]) # index of the last slot that can be used
values = np.array([2,5,7,3,4])
S = [np.array(s) for r in range(1, len(slots)+1) for s in itertools.combinations(range(len(slots)), r)]

# get function values (s1a1, s1a2, s1a3, s1a4, s1a5 s2a1, ...)
v = np.zeros(len(S)*len(agents)).astype(int)
for s_index,s in enumerate(S):
    for a in range(len(agents)):
        length_used = np.sum(s<=deadlines[a])
        v[s_index*len(agents)+a] = values[a] if length_used>=lengths[a] else 0
        
constraints_A = []
constraints_b = []
# set constraint that variables are positive
for c in range(len(S)*len(agents)):
    constraints_A.append([-1 if g==c else 0 for g in range(len(S)*len(agents))])
    constraints_b.append(0)
    
# set constraint that an agent can only have 1 set
for agent_index in range(len(agents)):
    constraints_A.append([1 if agent_index==a else 0 for c in range(len(S)) for a in range(len(agents))])
    constraints_b.append(1)

# finally, set the constraint that an item can only appear once in an assigned set
for j in range(len(slots)):
    sets_with_j = [s_index for s_index,s in enumerate(S) if j in s]
    constraints_A.append([1 if c in sets_with_j else 0 for c in range(len(S)) for a in range(len(agents))])
    constraints_b.append(1)

A = np.array(constraints_A)
b = np.array(constraints_b)

We can try a linear program solver, but unfortunately this is more difficult than the first assignment problem, and there is no guarantee of integer solutions:

In [3]:
res = linprog(-v, A_ub=A, b_ub=b)
for ind in np.where(res["x"]>0)[0]:
    s_index = ind//len(agents)
    a_index = ind%len(agents)
    print("weight",round(res["x"][ind],2),"slots", [slots[j] for j in S[s_index]], "agent", agents[a_index])

weight 0.33 slots ['10am'] agent E
weight 0.67 slots ['12pm'] agent E
weight 1.0 slots ['1pm'] agent C
weight 0.67 slots ['9am', '11am'] agent B
weight 0.33 slots ['10am', '12pm'] agent B
weight 0.33 slots ['9am', '10am', '11am'] agent D


You could maybe try and fix this (i.e., turn these partial fractions into integers), but it seems like integer programming (expensive, not guaranteed to be optimal) is the proper solution:

In [4]:
constraints = LinearConstraint(A, -np.inf, b)
res = milp(c=-v, integrality=np.ones_like(v), constraints=constraints)
for ind in np.where(res["x"]>0)[0]:
    s_index = ind//len(agents)
    a_index = ind%len(agents)
    print("weight",round(res["x"][ind],2),"slots", [slots[j] for j in S[s_index]], "agent", agents[a_index])

weight 1.0 slots ['12pm'] agent E
weight 1.0 slots ['1pm'] agent C
weight 1.0 slots ['9am', '11am'] agent B


This solution is the same as above, except that $B$ is doing 11AM instead of 10AM.

#### A more general form of competitive equilibrium

Lets define what competitive equilibrium means in this case. A solution $F$ is in competitive equilibrium for the prices $p$ if and only if:

- The given assignment at the given prices maximises each agents utility. I.e., $F_i = \text{argmax}_{T\subseteq X} (v_i(T)-\sum_{j|j\in T}p_j$)
- All time slots are at least the reserve price
- If a time slot is not assigned, it's value is the reserve price

As before, a solution is optimal if this is the case.

To prove this we first note the value of a solution is:

$$V(F)=\sum_{j|x_j\in F_\emptyset} q_j + \sum_{i\in N} v_i(F_i)$$

In words, we get the reserve price from any processors which aren't used, then the utility to each person (just the pure benefit of the combination). Applying the last bullet point above gives:

$$V(F)=\sum_{j|x_j\in F_\emptyset} p_j + \sum_{i\in N} v_i(F_i)$$

And then adding in (+1 ... -1) for all the used items, rearranging gives:

$$V(F)=\sum_{j|x_j\in X} p_j + \big[v_i(F_i)-\sum_{j|j\in F_i}p_j\big]$$

This quantity is guaranteed to be maximised given the price, as no individual agent can improve their utiltiy (bullet point 1). Nobody would switch their $F_i$, so the overall quantity must be at maximum.

Therefore, arriving at a competitive equilibrium means solving the problem.

Unfortunately, not all problems have a competitive equilibrium. Consider this case involving just the 9AM and 10AM slots, and where the reserve price is $\$3$:

$
\begin{array}{cclc}
\text{agent} & \text{length}(\lambda) & \text{deadline}(d) & \text{value}(w) \\
\hline
\text{A} & 2 & \text{11 AM} & \$10 \\
\text{B} & 1 & \text{11 AM} & \$6 \\
\end{array}
$

Consider whether Agent A has both slots or not:

1. Say agent A has both. Then the combined price must be below $\$10$, otherwise A would prefer to have nothing. Then at one of the two slots must be less than $\$5$. But agent B is willing to pay up to $\$6$ for a single slot, so this is not in equilibrium. Each agent must be assigned a set that maximises their return.
2. Say agent A has neither. Then the combined price must be greater than $\$10$, otherwise A would prefer to buy both. However, B is willing to pay at most $\$6$ for a slot, and the reserve price for the other is $\$3$, which is a total of $\$9$, which is less than $\$10$. So this is not in equilibrium either. 

This relates to a fundamental theorem:

**Theorem:** *This problem has a competitive equilibrium solution only if the corresponding linear program has an integer solution*

We can see our previous case is also not going to have one, as the linear solution gave some weights of 0.67 etc.

By looking at the conditions under which the linear program will have integer solutions we can also arrive at the following:

**Theorem:** *A competitive equilibrium will only occur for this problem if any of these options is met:*

- All agents want a single timeslot.
- Utility is additive across exclusive sets (i.e., $v_i(R)+v_i(T)=v_i(R\cup T)$.
- Time slots are substitutes and demand for one slot stays the same regardless of other prices.

#### An auction algorithm

Now we turn to finding an algorithm to arrive at a competitive equilibrium for the scheduling problem. For this we will use something called the ascending-auction algorithm. In this algorithm if an agent wants to aquire a slot they need to bid for it a fixed amount $\epsilon$ more than the person before. The algrotihm proceeds by figuring out an agents best option at the current prices, and updating the bids as it goes. For this let's consider another problem, one which does have a competitive equilibrium (and a linear program integer solution):

$
\begin{array}{cclc}
\text{agent} & \text{length}(\lambda) & \text{deadline}(d) & \text{value}(w) \\
\hline
\text{A} & 2 & \text{11 AM} & \$2 \\
\text{B} & 3 & \text{2 PM} & \$5 \\
\text{C} & 1 & \text{2 PM} & \$7 \\
\text{D} & 2 & \text{1 PM} & \$3 \\
\text{E} & 1 & \text{1 PM} & \$4 \\
\end{array}
$

One possible solution:

$
\begin{array}{lc}
\text{slot} & \text{user} \\
\hline
\text{9 AM} &  B \\
\text{10 AM} & B \\
\text{11 AM} & C \\
\text{12 PM} & E \\
\text{1 PM} &  B \\
\end{array}
$

In [5]:
slots_2 = ["9am","10am","11am","12pm","1pm"]
agents_2 = ["A","B","C","D","E"]
lengths_2 = np.array([2,3,1,2,1])
deadlines_2 = np.array([1,4,4,3,3]) # index of the last slot that can be used
values_2 = np.array([2,5,7,3,4])
S_2 = [np.array(s) for r in range(1, len(slots_2)+1) for s in itertools.combinations(range(len(slots_2)), r)]
assignments = []
epsilon = 0.25
price = np.zeros(len(slots_2))

for epoch in range(1000):
    any_change = False
    for a in range(len(agents_2)):
        p = price.copy()
        for s in range(len(slots_2)):
            if not (agents_2[a],slots_2[s]) in assignments:
                p[s]+=epsilon

        best_val = 0
        best_option = None
        for s_index,s in enumerate(S_2):
            length_used = np.sum(s<=deadlines_2[a])
            cost = np.sum(p[s])
            if length_used>=lengths_2[a]:
                val = values_2[a]-cost
                if(val>best_val):
                    best_val = val
                    best_option = s

        if not best_option is None:
            for s in best_option:
                if not (agents_2[a],slots_2[s]) in assignments:
                    price[s]+=epsilon
                    print(agents_2[a],"bids",price[s],"for",slots_2[s])
                    any_change = True
                    for other_a in range(len(agents_2)):
                        if (agents_2[other_a],slots_2[s]) in assignments:
                            assignments.remove((agents_2[other_a],slots_2[s]))
                    assignments.append((agents_2[a],slots_2[s]))
    
    if not any_change:
        print("done")
        break
    else:
        print("assignments:",assignments)
        print("prices:",dict(zip(slots_2,price)))

A bids 0.25 for 9am
A bids 0.25 for 10am
B bids 0.25 for 11am
B bids 0.25 for 12pm
B bids 0.25 for 1pm
C bids 0.5 for 9am
D bids 0.5 for 10am
D bids 0.5 for 11am
E bids 0.5 for 12pm
assignments: [('B', '1pm'), ('C', '9am'), ('D', '10am'), ('D', '11am'), ('E', '12pm')]
prices: {'9am': 0.5, '10am': 0.5, '11am': 0.5, '12pm': 0.5, '1pm': 0.25}
A bids 0.75 for 9am
A bids 0.75 for 10am
B bids 0.75 for 11am
B bids 0.75 for 12pm
C bids 0.5 for 1pm
D bids 1.0 for 9am
D bids 1.0 for 10am
E bids 1.0 for 11am
assignments: [('B', '12pm'), ('C', '1pm'), ('D', '9am'), ('D', '10am'), ('E', '11am')]
prices: {'9am': 1.0, '10am': 1.0, '11am': 1.0, '12pm': 0.75, '1pm': 0.5}
B bids 1.25 for 9am
B bids 0.75 for 1pm
C bids 1.0 for 12pm
D bids 1.25 for 11am
E bids 1.25 for 10am
assignments: [('B', '9am'), ('B', '1pm'), ('C', '12pm'), ('D', '11am'), ('E', '10am')]
prices: {'9am': 1.25, '10am': 1.25, '11am': 1.25, '12pm': 1.0, '1pm': 0.75}
B bids 1.25 for 12pm
C bids 1.0 for 1pm
D bids 1.5 for 9am
assignments: 

Unfortunately there is no guarantee this algorithm will converge. Sometimes the bidding increment is too small, other times an equilibrium can be missed. We also can't guarantee the solution found when the algorithm terminates is optimal, or even close to optimal. The upside is the algorithm is fairly cheap to run, and will always terminate.